In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logreg').getOrCreate()

In [2]:
from pyspark.ml.classification import LogisticRegression

In [3]:
df = spark.read.csv('titanic.csv', inferSchema=True, header=True)

In [4]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [6]:
my_col = df.select(['Survived', 'Sex', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'])

In [7]:
my_final_data = my_col.na.drop()

In [8]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer

In [9]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexInd')
gender_encoder = OneHotEncoder(inputCol='SexInd', outputCol='SexVec')

In [10]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkInd')
embark_encoder = OneHotEncoder(inputCol='EmbarkInd', outputCol='EmbarkVec')

In [11]:
assembler = VectorAssembler(inputCols=['Pclass', 'SexVec', 'EmbarkVec', 'Age', 'SibSp', 'Parch', 'Fare'],
                            outputCol='features')

In [12]:
from pyspark.ml import Pipeline

In [13]:
log_rig_ti = LogisticRegression(featuresCol='features', labelCol='Survived')

In [14]:
pipeline = Pipeline(stages=[gender_indexer, embark_indexer, gender_encoder, embark_encoder, assembler, log_rig_ti])

In [15]:
my_final_data.show()

+--------+------+------+----+-----+-----+-------+--------+
|Survived|   Sex|Pclass| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|  male|     3|22.0|    1|    0|   7.25|       S|
|       1|female|     1|38.0|    1|    0|71.2833|       C|
|       1|female|     3|26.0|    0|    0|  7.925|       S|
|       1|female|     1|35.0|    1|    0|   53.1|       S|
|       0|  male|     3|35.0|    0|    0|   8.05|       S|
|       0|  male|     1|54.0|    0|    0|51.8625|       S|
|       0|  male|     3| 2.0|    3|    1| 21.075|       S|
|       1|female|     3|27.0|    0|    2|11.1333|       S|
|       1|female|     2|14.0|    1|    0|30.0708|       C|
|       1|female|     3| 4.0|    1|    1|   16.7|       S|
|       1|female|     1|58.0|    0|    0|  26.55|       S|
|       0|  male|     3|20.0|    0|    0|   8.05|       S|
|       0|  male|     3|39.0|    1|    5| 31.275|       S|
|       0|female|     3|14.0|    0|    0| 7.8542|       

In [16]:
train_data, test_data = my_final_data.randomSplit([.7, .3])

In [17]:
fit_model = pipeline.fit(train_data)

In [18]:
result = fit_model.transform(test_data)

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [20]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [21]:
result.select('Survived', 'prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
+--------+----------+
only showing top 20 rows



In [22]:
AUC = my_eval.evaluate(result)

In [23]:
AUC

0.772541382667965